# Imaging and deconvolution demonstration

This script makes a fake data set and then deconvolves it. Finally the full and residual visibility are plotted.

In [1]:
%matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

results_dir = '/tmp/'

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u

from matplotlib import pyplot as plt


from rascil.data_models import PolarisationFrame

from rascil.processing_components import create_blockvisibility, show_image, export_image_to_fits, \
    deconvolve_cube, restore_cube, create_named_configuration, create_test_image, \
    create_image_from_visibility, advise_wide_field, invert_2d, predict_2d, \
    plot_uvcoverage, plot_visibility

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

mpl_logger = logging.getLogger("matplotlib") 
mpl_logger.setLevel(logging.WARNING) 

In [2]:
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

Construct LOW core configuration

In [3]:
lowr3 = create_named_configuration('LOWBD2', rmax=750.0)
print(lowr3)

create_named_configuration: LOWBD2
	(<Quantity -2565018.31203579 m>, <Quantity 5085711.90373391 m>, <Quantity -2861033.10788063 m>)
	GeodeticLocation(lon=<Longitude 116.76444824 deg>, lat=<Latitude -26.82472208 deg>, height=<Quantity 300. m>)
create_configuration_from_file: Maximum radius 750.0 m includes 236 antennas/stations
<xarray.Configuration>
Dimensions:   (id: 236, spatial: 3)
Coordinates:
  * id        (id) int64 0 1 2 3 4 5 6 7 8 ... 228 229 230 231 232 233 234 235
  * spatial   (spatial) <U1 'X' 'Y' 'Z'
Data variables:
    names     (id) <U10 'LOWBD2_0' 'LOWBD2_1' ... 'LOWBD2_270' 'LOWBD2_271'
    xyz       (id, spatial) float64 -1.697e+03 -4.573e+03 ... -4.728e+03 300.0
    diameter  (id) float64 35.0 35.0 35.0 35.0 35.0 ... 35.0 35.0 35.0 35.0 35.0
    mount     (id) <U2 'xy' 'xy' 'xy' 'xy' 'xy' ... 'xy' 'xy' 'xy' 'xy' 'xy'
    vp_type   (id) <U3 'LOW' 'LOW' 'LOW' 'LOW' 'LOW' ... 'LOW' 'LOW' 'LOW' 'LOW'
    offset    (id, spatial) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.

We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [4]:
times = numpy.zeros([1])
frequency = numpy.array([1e8])
channel_bandwidth = numpy.array([1e6])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox='J2000')
vt = create_blockvisibility(lowr3, times, frequency, channel_bandwidth=channel_bandwidth,
                       weight=1.0, phasecentre=phasecentre, polarisation_frame=PolarisationFrame('stokesI'))
print(vt)

create_blockvisibility: created 1 times
create_blockvisibility: 27966 rows, 0.002 GB
<xarray.BlockVisibility>
Dimensions:            (baselines: 27966, frequency: 1, polarisation: 1, spatial: 3, time: 1, uvw_index: 3)
Coordinates:
  * time               (time) float64 5.085e+09
  * baselines          (baselines) MultiIndex
  - antenna1           (baselines) int64 0 0 0 0 0 0 ... 233 233 233 234 234 235
  - antenna2           (baselines) int64 0 1 2 3 4 5 ... 233 234 235 234 235 235
  * frequency          (frequency) float64 1e+08
  * polarisation       (polarisation) <U1 'I'
  * uvw_index          (uvw_index) <U1 'u' 'v' 'w'
Dimensions without coordinates: spatial
Data variables:
    integration_time   (time) float32 1.0
    datetime           (time) datetime64[ns] 2020-01-01T10:31:38.475882830
    vis                (time, baselines, frequency, polarisation) complex128 ...
    weight             (time, baselines, frequency, polarisation) float32 0.0...
    imaging_weight     (time, ba

In [5]:
advice = advise_wide_field(vt, guard_band_image=3.0, delA=0.1, facets=1, wprojection_planes=1, 
                           oversampling_synthesised_beam=4.0)
cellsize = advice['cellsize']

advise_wide_field: (max_wavelength) Maximum wavelength 2.998 (meters)
advise_wide_field: (min_wavelength) Minimum wavelength 2.998 (meters)
advise_wide_field: (maximum_baseline) Maximum baseline 383.3 (wavelengths)
advise_wide_field: (maximum_w) Maximum w 125.8 (wavelengths)
advise_wide_field: (diameter) Station/dish diameter 35.0 (meters)
advise_wide_field: (primary_beam_fov) Primary beam 0.0857 (rad) 4.91 (deg) 1.77e+04 (asec)
advise_wide_field: (image_fov) Image field of view 0.257 (rad) 14.7 (deg) 5.3e+04 (asec)
advise_wide_field: (synthesized_beam) Synthesized beam 0.00261 (rad) 0.149 (deg) 538 (asec)
advise_wide_field: (cellsize) Cellsize 0.000652 (rad) 0.0374 (deg) 135 (asec)
advice_wide_field: (npixels) Npixels per side = 394
advice_wide_field: (npixels2) Npixels (power of 2) per side = 512
advice_wide_field: (npixels23) Npixels (power of 2, 3) per side = 512
advice_wide_field: (npixels_min) Npixels (power of 2, 3, 4, 5) per side = 512
advice_wide_field: (w_sampling_image) W sa

Plot the synthesized uv coverage.

In [6]:
plt.clf()
plot_uvcoverage([vt])
plt.show()

IndexError: index 1 is out of bounds for axis 4 with size 1

<Figure size 864x864 with 0 Axes>

Read the venerable test image, constructing an image

In [ ]:
m31image = create_test_image(frequency=frequency, cellsize=cellsize,
                             phasecentre=phasecentre)
nchan, npol, ny, nx = m31image["pixels"].data.shape

fig=show_image(m31image)

In [ ]:
vt = predict_2d(vt, m31image, context='2d')

plt.clf()
plot_visibility([vt])
plt.show()

Make the dirty image and point spread function

In [ ]:
model = create_image_from_visibility(vt, cellsize=cellsize, npixel=512)
dirty, sumwt = invert_2d(vt, model, context='2d')
psf, sumwt = invert_2d(vt, model, context='2d', dopsf=True)

show_image(dirty)
print("Max, min in dirty image = %.6f, %.6f, sumwt = %f" % (dirty["pixels"].data.max(), dirty["pixels"].data.min(), sumwt))

print("Max, min in PSF         = %.6f, %.6f, sumwt = %f" % (psf["pixels"].data.max(), psf["pixels"].data.min(), sumwt))

export_image_to_fits(dirty, '%s/imaging_dirty.fits'%(results_dir))
export_image_to_fits(psf, '%s/imaging_psf.fits'%(results_dir))

Deconvolve using clean

In [ ]:
comp, residual = deconvolve_cube(dirty, psf, niter=10000, threshold=0.001, fractional_threshold=0.001,
                                 window_shape='quarter', gain=0.7, scales=[0, 3, 10, 30])

restored = restore_cube(comp, psf, residual)

# Show the results

fig=show_image(comp)
plt.title('Solution')
fig=show_image(residual)
plt.title('Residual')
fig=show_image(restored)
plt.title('Restored')

print(restored)

Predict the visibility of the model

In [ ]:
vtmodel = create_blockvisibility(lowr3, times, frequency, channel_bandwidth=channel_bandwidth,
                            weight=1.0, phasecentre=phasecentre, 
                            polarisation_frame=PolarisationFrame('stokesI'))
vtmodel=predict_2d(vtmodel, comp, context='2d')

Now we will plot the original visibility and the residual visibility.

In [ ]:
vtmodel.vis.values = vt.vis.values - vtmodel.vis.values

plt.clf()
plot_visibility([vt, vtmodel], colors=['b', 'r'])
plt.show()